# Whitening transform
This notebook explores the effects of the *whitening transform*
(and its subroutine, the *Cholesky decomposition*)
on a few isolated example inputs.

### Cholesky decomposition
Decompose a matrix $M$ into two matrices $W$ of the same shape,
such that
$M = WW^{\top}$.

Note that $M$ must be
1. Symmetric
2. Positive definite
(In the example below, a covariance matrix is used.
These are always symmetric and positive definite.)

Also, $W$ is always lower triangular (i.e. entries
above the main diagonal are always 0).

It is supposedly faster than the Eigenvalue-decomposition.
PyTorch implements both only on the CPU,
so I doubt either is particularly fast to use in Neural Networks.

In [2]:
import torch
import random
from rolim.tools.stats import sample_covar
from rolim.tools.testing import assert_tensor_eq

shape = (3, 10)
elements = [x for x in range(shape[0]*shape[1])]
random.shuffle(elements)
sample = torch.tensor(elements, dtype=torch.float).reshape(shape)
m = sample_covar(sample)
m


tensor([[ 39.8333,  16.9444,  20.2778],
        [ 16.9444,  94.9889, -43.5444],
        [ 20.2778, -43.5444, 114.5444]])

In [3]:
w = torch.linalg.cholesky(m)
print(w)
print(w @ w.T)
torch.allclose(m, w@w.T)

tensor([[ 6.3114,  0.0000,  0.0000],
        [ 2.6848,  9.3692,  0.0000],
        [ 3.2129, -5.5683,  8.5566]])
tensor([[ 39.8333,  16.9444,  20.2778],
        [ 16.9444,  94.9889, -43.5444],
        [ 20.2778, -43.5444, 114.5445]])


True

Note that $W$ is always lower-triangular.
So we can use torch's dedicated `torch.triangular_solve()`:

In [10]:
output = torch.triangular_solve(torch.eye(w.shape[0]), w)
print(output)
output[0] @ w

torch.return_types.triangular_solve(
solution=tensor([[0.1584, 0.0000, 0.0000],
        [0.0000, 0.1067, 0.0000],
        [0.0000, 0.0000, 0.1169]]),
cloned_coefficient=tensor([[ 6.3114,  0.0000,  0.0000],
        [ 2.6848,  9.3692,  0.0000],
        [ 3.2129, -5.5683,  8.5566]]))


tensor([[ 1.0000,  0.0000,  0.0000],
        [ 0.2866,  1.0000,  0.0000],
        [ 0.3755, -0.6508,  1.0000]])

The above does not seem very accurate.
`torch.linalg.inv()` does better:

In [11]:
torch.linalg.inv(w) @ w

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 2.5107e-08, -2.8204e-08,  1.0000e+00]])